In [1]:
import pandas as pd
import numpy as np
import zipfile
#import seaborn as sns
#import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from torch import nn
import torchvision
from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from PIL import Image
import os
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import time 
from torch.optim import lr_scheduler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
PATH = '/content/drive/MyDrive/Colab Notebooks/Santa/train.zip'
DIR = '/content/drive/MyDrive/Colab Notebooks/Santa'
TRAIN_DATASET= '/content/drive/MyDrive/Colab Notebooks/Santa/train'
MODEL_WEIGHTS = "/content/drive/MyDrive/Colab Notebooks/Santa/"
TRAIN_CSV = '/content/drive/MyDrive/Colab Notebooks/Santa/train.csv'
TEST_DIR = '/content/drive/MyDrive/Colab Notebooks/Santa/test'
OUT = '/content/drive/MyDrive/Colab Notebooks/Santa/out/'
RESCALE_SIZE = 380
CROP_SIZE = 250
use_gpu = torch.cuda.is_available()
torch.manual_seed(210122)
use_gpu

True

In [4]:
#with zipfile.ZipFile(PATH, 'r') as zip_ref:
#    zip_ref.extractall(DIR)

In [5]:
#датасет из бейзлайна соревнования
class DedyDataset(Dataset):
    def __init__(self, root_dir, csv_path=None, transform=None):
        
        self.transform = transform
        self.files = [os.path.join(root_dir, fname) for fname in os.listdir(root_dir)]
        self.targets = None
        if csv_path:
            df = pd.read_csv(csv_path, sep="\t")
            self.targets = df["class_id"].tolist()
            self.files = [os.path.join(root_dir, fname) for fname in df["image_name"].tolist()]

    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        image = Image.open(self.files[idx])
        target = self.targets[idx] if self.targets else -1
        if self.transform:
            image = self.transform(image)
        return image, target

In [6]:
#для отображения картинок из датасета
def imshow(inp, title=None):
    """Imshow для тензоров"""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.set_title(title)
    plt.grid(False)
import matplotlib.pyplot as plt

In [7]:
#трансформер для апсемплинга и баллансировки картинок
producer = T.Compose([    T.RandomHorizontalFlip(p=0.8),
                          T.RandomRotation(degrees=(-25,25)),
                          T.RandomPerspective(distortion_scale=0.2, p=0.2),
                          T.ColorJitter(brightness=0.1, saturation=0.1, contrast=0.1),
                          T.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
                          T.RandomAffine(degrees=(0,0), translate=(0.1, 0.3)),
                          T.RandomAutocontrast(p=0.3)])
prodset = DedyDataset(TRAIN_DATASET, TRAIN_CSV, producer)

In [8]:
#начальное состояние датасета
train_df = pd.read_csv(TRAIN_CSV, sep='\t')
train_df.shape

(14400, 2)

In [9]:
#проверка на соответствие размеров
assign(len(prodset)==train_df.shape[0])

14400

In [ ]:
#классы не сбаллансированы
train_df.value_counts()

In [10]:
#баллансировка
for i,img in enumerate(tqdm(prodset)):
  for cnt in range(0,2):
    if img[1]!=0:
      train_df=train_df.append({'image_name':str(cnt+2)+'_'+prodset.files[i].split('/')[-1], 'class_id':img[1]}, ignore_index=True)
      img[0].save(TRAIN_DATASET+'/'+str(cnt+2)+'_'+prodset.files[i].split('/')[-1])
#увеличим датасет 
for i,img in enumerate(tqdm(prodset)):
  for cnt in range(0,2):
      train_df=train_df.append({'image_name':str(cnt+4)+'_'+prodset.files[i].split('/')[-1], 'class_id':img[1]}, ignore_index=True)
      img[0].save(TRAIN_DATASET+'/'+str(cnt+4)+'_'+prodset.files[i].split('/')[-1])

train_df.shape


In [11]:
#обновим CSV
train_df.to_csv(TRAIN_CSV, sep='\t', index=False)

In [12]:
#модель Efficientnet_B4 +полноразмерные слои с дропаутом и нелинейностью
def init_model(t):
  model = torchvision.models.efficientnet_b4(pretrained=True)
  model.classifier = nn.Sequential(nn.Dropout(p=0.5, inplace=True),
                                   nn.Linear(in_features=1792, out_features=896, bias=True),
                                   nn.Dropout(p=0.5, inplace=True),
                                   nn.SELU(),
                                   nn.Linear(in_features=896, out_features=448, bias=True),
                                   nn.Dropout(p=0.5, inplace=True),
                                   nn.SELU(),
                                   nn.Linear(in_features=448, out_features=224, bias=True),
                                   nn.Dropout(p=0.5, inplace=True),
                                   nn.Linear(in_features=224, out_features=3, bias=True))
  #подгрузка весов, если упало по таймауту колаба
  #model.load_state_dict(torch.load(MODEL_WEIGHTS+'efficiantnet_b4_fine_tune_need_more_train.pth'))
  # Использовать ли GPU
  if t:
    model = model.cuda()
  return model

In [13]:
def train_model(model, dataloaders, criterion, optimizer, scheduler,
                phases, num_epochs=25):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0
    best_f1 = 0.0
    losses = {'train': [], "val": []}

    max_useless_epoch = 0
    torch.cuda.empty_cache()
    for epoch in tqdm(range(0,num_epochs)):

        # каждя эпоха имеет обучающую и тестовую стадии
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # установаить модель в режим обучения
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            running_labels = torch.tensor([])
            running_preds = torch.tensor([])
            # итерируемся по батчам
            for data in dataloaders[phase]:
                # получаем картинки и метки
                inputs, labels = data

                # оборачиваем в переменные
                if use_gpu:
                    inputs = inputs.cuda()
                    labels = labels.cuda()
                else:
                    inputs, labels = inputs, labels

                # инициализируем градиенты параметров
                if phase == 'train':
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                else:
                    with torch.no_grad():
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)    
                preds = torch.argmax(outputs, -1)
               

                # backward pass + оптимизируем только если это стадия обучения
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # статистика
                running_loss += loss.item()
                running_corrects += int(torch.sum(preds == labels.data))
                running_labels = torch.cat([running_labels,labels.data.cpu()], 0)
                running_preds = torch.cat([running_preds,preds.cpu()], 0)
                
            epoch_loss = running_loss /  len(dataloaders[phase].dataset)
            epoch_acc = running_corrects / len(dataloaders[phase].dataset)

            epoch_f1 = f1_score(running_labels.int().numpy(),running_preds.int().numpy(), average='weighted')

            # Ваш код здесь
            losses[phase].append(epoch_loss)
            
            print('{} Loss: {:.4f} Acc: {:.4f} F1 {:.4f}'.format(
                                    phase, epoch_loss, epoch_acc, epoch_f1
                                ))

            # если достиглось лучшее качество, то запомним веса модели
            if phase == 'val':
              if epoch_f1 > best_f1:
                best_acc = epoch_acc
                best_f1 = epoch_f1
                best_model_wts = model.state_dict()
                torch.save(model.state_dict(), MODEL_WEIGHTS+'efficiantnet_b4_fine_tune_need_more_train.pth')
                max_useless_epoch=0  
                if best_f1==1:
                  max_useless_epoch=20
              else:
                max_useless_epoch+=1    
            torch.cuda.empty_cache()
        if max_useless_epoch>=20:
          break
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_f1))

    # загрузим лучшие веса модели
    model.load_state_dict(best_model_wts)
    return model, best_acc

In [14]:
if __name__ == "__main__":
    torch.cuda.empty_cache()
    #добавим еще чуть инстант аугментации
    transformer = T.Compose([T.Resize(size=(RESCALE_SIZE,RESCALE_SIZE)),
                          T.RandomHorizontalFlip(p=0.5),
                          T.RandomRotation(degrees=(-10,10)),
                          T.RandomPerspective(distortion_scale=0.1, p=0.1),
                          T.ColorJitter(brightness=0.1, saturation=0.1, contrast=0.1),
                          T.ToTensor(),
                          T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    dset = DedyDataset(TRAIN_DATASET, TRAIN_CSV, transformer)
    labels = dset.targets
    indices = list(range(len(labels)))
    ind_train, ind_test, _, _ = train_test_split(indices, labels, test_size=0.2, random_state=200122, stratify=labels)

    trainset = torch.utils.data.Subset(dset, ind_train)                           
    testset = torch.utils.data.Subset(dset, ind_test)

    batch_size = 16
    num_workers = 0
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=num_workers)

    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                            shuffle=False, num_workers=num_workers)

    loaders = {'train': trainloader, 'val': testloader}


    model = init_model(use_gpu)
    
    loss = nn.CrossEntropyLoss() #функция потерь
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) #оптимизатор
    scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5) #уменьшает learning rate каждые 5 эпох

    criterion = nn.CrossEntropyLoss()

    # Train
    train_results = train_model(model, loaders, criterion, optimizer,scheduler, phases=['train', 'val'], num_epochs=100)


Downloading: "https://download.pytorch.org/models/efficientnet_b4_rwightman-7eb33cd5.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b4_rwightman-7eb33cd5.pth


  0%|          | 0.00/74.5M [00:00<?, ?B/s]

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


train Loss: 0.0128 Acc: 0.9202 F1 0.9197
val Loss: 0.0015 Acc: 0.9927 F1 0.9927


  1%|          | 1/100 [16:39<27:29:58, 999.99s/it]

...

val Loss: 0.0002 Acc: 0.9993 F1 0.9993
train Loss: 0.0006 Acc: 0.9969 F1 0.9969


  5%|▌         | 5/100 [1:10:41<21:48:46, 826.59s/it]

val Loss: 0.0000 Acc: 0.9997 F1 0.9997
train Loss: 0.0004 Acc: 0.9980 F1 0.9980
val Loss: 0.0000 Acc: 1.0000 F1 1.0000


  5%|▌         | 5/100 [1:24:10<26:39:20, 1010.11s/it]

Training complete in 84m 11s
Best val Acc: 1.000000


In [15]:
torch.save(model.state_dict(), MODEL_WEIGHTS+'efficiantnet_b4_fine_tune8_last.pth')

In [ ]:
#предсказание
def predict(model, dataloader):
    with torch.no_grad():
        logits = []
        labels = []
        samples = []
        # итерируемся по батчам
        for i, data in enumerate(dataloader):
            inputs, label = data
            if use_gpu:
                inputs = inputs.cuda()
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)
            labels.append(label)
    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    labels = torch.cat(labels)
    return probs, labels

In [ ]:
test_transformer = T.Compose([T.Resize((RESCALE_SIZE,RESCALE_SIZE)),
                              T.ToTensor(),
                              T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
batch_size = 16
test_images  = DedyDataset(TEST_DIR,None, test_transformer)
test_loader = DataLoader(test_images, batch_size=batch_size)
#эта часть была в другом модуле, так что сохраненные веса загружались отдельно
#model = init_model(use_gpu)
#model.load_state_dict(torch.load(MODEL_WEIGHTS+'efficiantnet_b4_fine_tune.pth'))
prediction, labels = predict(model,test_loader)

In [ ]:
test_filenames = [path.split('/')[-1] for path in test_images.files]
preds = np.argmax(prediction,-1)
test_filenames

In [ ]:
#выходной файл сабмита
df = pd.DataFrame()
df['Id'] = test_filenames
df['Expected'] = preds
df.to_csv(OUT+'submission.csv', index=False, header=False, sep='\t' )